In [1]:
import seaborn as sns
sns.set()

In [2]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_parquet('Fragrance_database.parquet')
df.head()

,fragrance,brand,gender,year,rating,count_ratings,accords,accord_opacity,accord_width,votes_have,...,pro_net_votes,con_net_votes,comment_users,comment_date,comments,url,filename,file_size,summary_reviews,joined_comments
0,Hacivat X,Nishane,women and men,2023.0,4.00,18.0,"[citrus, woody, white floral, aromatic, sweet,...","[1.0, 0.817, 0.79, 0.652, 0.6355, 0.574, 0.5455]","[1.0, 0.7865000000000001, 0.755, 0.594, 0.5747...",0.051136,...,[],[],"[Patj1994, theperfumequeentiktok, NERI12, Yalc...","[datetime.datetime(2023, 5, 1, 0, 0), datetime...",[This looks promising from reading the notes. ...,https://www.fragrantica.com/perfume/Nishane/Ha...,_Nishane-Hacivat X.pkd,0.591532,None,this looks promising from reading the notes. i...
1,Cool Water Woman Sea Rose Pacific Summer Edition,Davidoff,women,2017.0,4.08,37.0,"[woody, rose, fruity, citrus, sweet, musky, po...","[1.0, 0.986466, 0.870301, 0.812782, 0.75188, 0...","[1.0, 0.984211, 0.848684, 0.781579, 0.710527, ...",0.411765,...,[],[],[LillyGill],"[datetime.datetime(2017, 8, 10, 0, 0)]","[First of all, the bottles in the Cool Water W...",https://www.fragrantica.com/perfume/Davidoff/C...,_Davidoff-Cool Water Woman Sea Rose Pacific Su...,0.353396,None,"first of all, the bottles in the cool water wo..."
2,Trésor,Lancôme,women,1990.0,3.64,11190.0,"[fruity, powdery, floral, sweet, rose, vanilla...","[1.0, 0.962732, 0.824044, 0.74249, 0.666979, 0...","[1.0, 0.95652, 0.7947190000000001, 0.699572, 0...",0.436839,...,"[30, 27, 27, 25, 22, 12, 4, -11]","[18, 10, 13, 4, -4, -4, -10, -19]","[Gemsense, jungirl, luna k, Rose1412, CyberMer...","[datetime.datetime(2023, 5, 23, 0, 0), datetim...",[The notes sound like this will smell amazing ...,https://www.fragrantica.com/perfume/Lancome/Tr...,_Lancome-Tresor.pkd,2.459217,has sentimental value and can evoke fond memor...,the notes sound like this will smell amazing a...
3,The Collection Magnetic Musk,Hugo Boss,men,2021.0,4.25,12.0,"[beeswax, iris, powdery, honey, aromatic, vani...","[1.0, 1.0, 0.9475, 0.85, 0.8425, 0.7675, 0.745...","[1.0, 1.0, 0.93875, 0.825, 0.81625, 0.72875, 0...",0.083333,...,[],[],"[eaudemale, Ramstiam, fragrance_girl, fragranc...","[datetime.datetime(2023, 3, 23, 0, 0), datetim...","[Waxy, honey, warm, nutty and slightly sweet. ...",https://www.fragrantica.com/perfume/Hugo-Boss/...,_Hugo-Boss-The Collection Magnetic Musk.pkd,0.851752,None,"waxy, honey, warm, nutty and slightly sweet. i..."
4,CK One Summer 2017,Calvin Klein,women and men,2017.0,3.57,168.0,"[citrus, warm spicy, ozonic, fresh spicy, aqua...","[1.0, 0.776875, 0.70375, 0.696953, 0.67, 0.599...","[1.0, 0.739687, 0.654375, 0.6464449999999999, ...",0.514493,...,[],[],"[cm66, GodKing, Petals_and_Perfumes, giuseppe9...","[datetime.datetime(2023, 4, 16, 0, 0), datetim...",[Don't understand the hate on this one? Year r...,https://www.fragrantica.com/perfume/Calvin-Kle...,_Calvin-Klein-CK One Summer 2017.pkd,0.628683,None,dont understand the hate on this one? year rou...


In [4]:
df = df.astype({'fragrance':'string'})

In [473]:
def remove_multistrings(cur_string, replace_list):
    for cur_word in replace_list:
        cur_string = cur_string.replace(cur_word, '')
    return cur_string

In [5]:
df.columns

Index(['fragrance', 'brand', 'gender', 'year', 'rating', 'count_ratings',
       'accords', 'accord_opacity', 'accord_width', 'votes_have', 'votes_had',
       'votes_want', 'votes_love', 'votes_like', 'votes_ok', 'votes_dislike',
       'votes_hate', 'votes_winter', 'votes_spring', 'votes_summer',
       'votes_fall', 'votes_day', 'votes_night', 'top_notes', 'middle_notes',
       'base_notes', 'overall_notes', 'note_pyramid', 'perfumers', 'longevity',
       'sillage', 'masculine', 'value', 'fragrance_line', 'similar', 'pros',
       'cons', 'pro_net_votes', 'con_net_votes', 'comment_users',
       'comment_date', 'comments', 'url', 'filename', 'file_size',
       'summary_reviews', 'joined_comments'],
      dtype='object')

In [6]:
features = ['votes_summer','votes_day' ,'longevity','sillage','summary_reviews'] # 'masculine'
df_1 = df[df['masculine'] > 3]
df_1 = df_1[['fragrance','brand']+features].dropna()
len(df_1)

1019

In [8]:
features = ['votes_summer','votes_day' ,'longevity','sillage','joined_comments']
df_2 = df[df['masculine'] > 3]
df_2 = df_2[['fragrance','brand']+features].dropna()
len(df_2)

1940

In [15]:
# from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.01, max_df = 0.9, ngram_range=(1,4), stop_words='english') #max_features=4000

In [18]:
input_matrix = vectorizer.fit_transform(df.dropna(subset = ['joined_comments'])['joined_comments'])
vectorizer.get_feature_names_out()

array(['00', '000', '00s', ..., 'zyb', 'zyd', 'zydy'], dtype=object)

In [36]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [remove_stopwords, 
                      strip_punctuation, 
                      strip_short, 
                      stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

In [37]:
df['processed_comments'] = df['joined_comments'].fillna('').apply(lambda x: preprocess(x))

In [41]:
from gensim import corpora
dictionary = corpora.Dictionary(corpus)

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [45]:
gender = [('Female',df['masculine'] < 2.5), ('Male',df['masculine'] > 3.5), ('Unisex',(df['masculine'] > 2) & (df['masculine'] < 4))]
weather = [('Hot',df['votes_summer'] > df['votes_winter']), ('Cold',df['votes_summer'] < df['votes_winter']), ('Year-Round',(df['votes_summer'] > 0.4) & (df['votes_winter'] > 0.4))]
time_day = [('Day',df['votes_day'] > df['votes_night'] - 0.2), ('Night',df['votes_day'] - 0.2 < df['votes_night']), ('Any',(df['votes_day'] > 0.4) & (df['votes_night'] > 0.4))]

In [54]:
dict_models = dict()
for cond_gender in gender:
    for cond_weather in weather:
        for cond_day in time_day:
            df_filtered = df[cond_gender[1] & cond_weather[1] & cond_day[1]]

            # create a dictionary with the corpus
            corpus = df_filtered['processed_comments']

            # convert corpus into a bag of words
            bow = [dictionary.doc2bow(text) for text in corpus]
            
            # find the coherence score with a different number of topics
            max_coh_score = 0
            for i in range(2,15):
                lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
                coherence_model = CoherenceModel(model=lsi, texts=df_filtered['processed_comments'], dictionary=dictionary, coherence='c_v')
                coherence_score = coherence_model.get_coherence()
                if coherence_score > max_coh_score:
                    max_coherent = i
                    
            # perform SVD on the bag of words with the LsiModel to extract topics
            lsi = LsiModel(bow, num_topics=max_coherent, id2word=dictionary)
            
            dict_models[(cond_gender[0],cond_weather[0],cond_day[0])] = lsi

/home/felipemazin/.local/lib/python3.8/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/home/felipemazin/.local/lib/python3.8/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [58]:
lsi_bla = dict_models[('Male','Hot','Day')]

In [71]:
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation,strip_numeric

lsi_bla_topics = lsi_bla.show_topics(num_words=15)

topics = []
filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]

for topic in lsi_bla_topics:
    print(topic)
    topics.append(preprocess_string(topic[1], filters))

print(topics)

(0, '0.374*"fragranc" + 0.366*"smell" + 0.332*"like" + 0.319*"scent" + 0.174*"note" + 0.152*"fresh" + 0.145*"good" + 0.126*"love" + 0.124*"wear" + 0.117*"bottl" + 0.114*"hour" + 0.099*"dont" + 0.096*"citru" + 0.093*"open" + 0.093*"time"')
(1, '-0.811*"vetiv" + -0.196*"guerlain" + 0.190*"smell" + -0.144*"citru" + 0.139*"blue" + 0.105*"aventu" + 0.087*"aquat" + -0.078*"grei" + -0.077*"terr" + -0.076*"eau" + -0.071*"green" + -0.068*"classic" + -0.064*"earthi" + -0.060*"origin" + 0.059*"sweet"')
(2, '0.360*"homm" + 0.352*"dior" + 0.329*"sport" + 0.297*"citru" + -0.263*"vetiv" + 0.191*"lemon" + 0.189*"cologn" + 0.172*"summer" + -0.171*"smell" + -0.166*"aventu" + -0.144*"like" + 0.143*"version" + 0.136*"eau" + -0.133*"green" + 0.123*"iri"')
(3, '-0.702*"aventu" + -0.220*"creed" + 0.182*"mint" + -0.149*"pineappl" + 0.147*"blue" + 0.136*"fresh" + -0.130*"batch" + 0.129*"aquat" + -0.127*"dior" + 0.113*"summer" + -0.106*"leather" + -0.102*"clone" + -0.101*"cologn" + 0.094*"green" + -0.092*"price

In [60]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi_bla.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.374*"fragranc" + 0.366*"smell" + 0.332*"like" + 0.319*"scent" + 0.174*"note" + 0.152*"fresh" + 0.145*"good" + 0.126*"love" + 0.124*"wear" + 0.117*"bottl".
Words in 1: -0.811*"vetiv" + -0.196*"guerlain" + 0.190*"smell" + -0.144*"citru" + 0.139*"blue" + 0.105*"aventu" + 0.087*"aquat" + -0.078*"grei" + -0.077*"terr" + -0.076*"eau".
Words in 2: 0.360*"homm" + 0.352*"dior" + 0.329*"sport" + 0.297*"citru" + -0.263*"vetiv" + 0.191*"lemon" + 0.189*"cologn" + 0.172*"summer" + -0.171*"smell" + -0.166*"aventu".
Words in 3: -0.702*"aventu" + -0.220*"creed" + 0.182*"mint" + -0.149*"pineappl" + 0.147*"blue" + 0.136*"fresh" + -0.130*"batch" + 0.129*"aquat" + -0.127*"dior" + 0.113*"summer".
Words in 4: 0.305*"blue" + 0.299*"origin" + 0.253*"vetiv" + -0.240*"scent" + 0.195*"aquat" + 0.174*"summer" + -0.168*"fragranc" + 0.134*"grapefruit" + -0.127*"masculin" + -0.125*"floral".
Words in 5: 0.409*"mint" + -0.282*"blue" + -0.230*"fragranc" + 0.185*"aventu" + -0.183*"vetiv" + 0.182*"note" + 0.

In [24]:
topic_word_list = []
def get_topics(components):
    for i, comp in enumerate(components):
        terms_comp = zip(vocab,comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic=" "
        for t in sorted_terms:
            topic= topic + ' ' + t[0]
            topic_word_list.append(topic)
    return topic_word_list

get_topics(components)

['  scent',
 '  scent fragrance',
 '  scent fragrance smell',
 '  scent fragrance smell just',
 '  scent fragrance smell just love',
 '  scent fragrance smell just love sweet',
 '  scent fragrance smell just love sweet perfume',
 '  rose',
 '  rose oud',
 '  rose oud leather',
 '  rose oud leather roses',
 '  rose oud leather roses aoud',
 '  rose oud leather roses aoud rose oud',
 '  rose oud leather roses aoud rose oud montale',
 '  rose',
 '  rose floral',
 '  rose floral perfume',
 '  rose floral perfume roses',
 '  rose floral perfume roses love',
 '  rose floral perfume roses love jasmine',
 '  rose floral perfume roses love jasmine fruity',
 '  vanilla',
 '  vanilla sweet',
 '  vanilla sweet tuberose',
 '  vanilla sweet tuberose leather',
 '  vanilla sweet tuberose leather perfume',
 '  vanilla sweet tuberose leather perfume jasmine',
 '  vanilla sweet tuberose leather perfume jasmine ylang',
 '  oud',
 '  oud floral',
 '  oud floral tuberose',
 '  oud floral tuberose jasmine',


In [25]:
ls = get_topics(components)

In [27]:
len(ls)

560

In [525]:
scaler = StandardScaler()
nn1 = NearestNeighbors(n_neighbors=10)
nn1.fit(X)

NearestNeighbors(n_neighbors=10)

In [547]:
df_1.iloc[847,]

fragrance                                      Leather Eau de Parfum
brand                                                 Acqua di Parma
votes_summer                                                0.153465
votes_day                                                   0.475248
longevity                                                   3.762646
sillage                                                     3.430034
summary_reviews    intense  note unique and satisfying scent eleg...
Name: 5154, dtype: object

In [548]:
dists, indices = nn1.kneighbors(X[847])
df_1.iloc[indices[0]]

,fragrance,brand,votes_summer,votes_day,longevity,sillage,summary_reviews
5154,Leather Eau de Parfum,Acqua di Parma,0.153465,0.475248,3.762646,3.430034,intense note unique and satisfying scent eleg...
3747,B-612,Nishane,0.625000,0.892361,3.710106,3.239397,"has a calming, serene quality pleasant and uni..."
841,Ombre Leather 16,Tom Ford,0.153333,0.773333,3.905172,3.380178,lighter version of tuscan with excellent long...
1529,Gucci Guilty Intense Pour Homme,Gucci,0.356125,0.652422,3.334029,2.983212,great scent for those who want to smell good a...
74,Morrocan Leather,Memo Paris,0.258929,0.830357,3.648855,3.226010,"not overly y, no tar or birch heaviness fresh ..."
3454,Dior Homme Intense 2011,Dior,0.146979,0.406372,3.969940,3.343426,elegant and distinguished scent good winter ev...
4450,Vibrant Leather Oud,Zara,0.474026,0.859307,2.825746,2.563056,affordable fragrance elegant masculine scent g...
3766,Side Effect,Initio Parfums Prives,0.122201,0.369802,4.131767,3.716937,intoxicating fragrance good quality perfume ca...
4672,Royal Mayfair,Creed,0.753876,0.937984,3.338505,2.938770,natural and beautiful eucalyptus note good bal...
2439,Bvlgari Man Wood Neroli,Bvlgari,1.000000,0.953488,3.134220,2.853289,clean and fresh scent uplifting and uplifting ...


In [528]:
X2 = vectorizer.fit_transform(df_2['all_comments'])
vectorizer.get_feature_names_out()

array(['00', '10', '10 10', ..., 'zesty', 'zingy', 'zino'], dtype=object)

In [529]:
nn2 = NearestNeighbors(n_neighbors=10)
nn2.fit(X2)

NearestNeighbors(n_neighbors=10)

In [530]:
df_2.iloc[367,]

fragrance                                         The Most Wanted
brand                                                      Azzaro
votes_summer                                             0.143108
votes_day                                                0.368745
longevity                                                3.791563
sillage                                                  3.236853
all_comments    smells like sexy grape soda, would buy 8/10 do...
Name: 2312, dtype: object

In [531]:
dists, indices = nn2.kneighbors(X2[367])
df_2.iloc[indices[0]]

,fragrance,brand,votes_summer,votes_day,longevity,sillage,all_comments
2312,The Most Wanted,Azzaro,0.143108,0.368745,3.791563,3.236853,"smells like sexy grape soda, would buy 8/10 do..."
3321,L'Homme Parfum Intense,Yves Saint Laurent,0.201024,0.628681,3.334719,2.883936,"this was my first ever fragrance, bought in fr..."
2593,Euphoria Men Intense,Calvin Klein,0.195489,0.532331,3.096862,2.744431,i loved the original for several years. i fina...
4713,The One Luminous Night,Dolce&Gabbana,0.129577,0.400000,3.732240,3.008475,first impression: \nthis is within realm of l...
5198,Euphoria Men,Calvin Klein,0.604608,1.000000,2.485623,2.188200,shaving cream. a whiff of ginger if i squint a...
1550,Legend Night,Montblanc,0.203733,0.412131,2.924224,2.694058,i really like how this smells but unfortunatel...
341,L'Eau d'Issey Pour Homme Noir Ambre,Issey Miyake,0.099391,0.405680,3.841837,3.327922,i bought into the hype & after a few months of...
1410,The One Gentleman,Dolce&Gabbana,0.300000,1.000000,3.165217,2.709615,@dolcegabbana . \nwhen flanker is as good o...
4330,Wanted,Azzaro,0.512069,0.946552,3.641686,3.238205,"its alright, smells similarly to a mix of ince..."
3432,African Leather,Memo Paris,0.225677,0.664995,3.715311,3.228522,"thank god, i get no cumin at all. this is a sp..."


In [488]:
df_2.iloc[36,]

fragrance                            Aqva Pour Homme Marine Toniq
brand                                                     Bvlgari
votes_summer                                             0.976563
votes_day                                                     1.0
longevity                                                3.377778
sillage                                                  3.169355
all_comments    If you can't afford Oud Minerale, then get the...
Name: 262, dtype: object

In [537]:
df_2.iloc[263,]

fragrance                                  Acqua di Parma Colonia
brand                                              Acqua di Parma
votes_summer                                                  1.0
votes_day                                                0.994714
longevity                                                2.790397
sillage                                                  2.562204
all_comments    i have collected over 200 fragrances and today...
Name: 1741, dtype: object

In [538]:
# x2 = vectorizer.transform()
dists, indices = nn2.kneighbors(X2[263])
df_2.iloc[indices[0]]

,fragrance,brand,votes_summer,votes_day,longevity,sillage,all_comments
1741,Acqua di Parma Colonia,Acqua di Parma,1.000000,0.994714,2.790397,2.562204,i have collected over 200 fragrances and today...
4127,Colonia Intensa,Acqua di Parma,0.929059,1.000000,3.141968,2.728947,"it was not love at first sniff, but has worke..."
1924,Eau Sauvage,Dior,0.955891,1.000000,2.810244,2.525920,nothing more can be said about this goatiest o...
4859,Neroli Sauvage,Creed,1.000000,0.938053,2.650909,2.592183,"i don't get much at all, mostly citrus only. ..."
4853,Bois du Portugal,Creed,0.344086,0.912442,3.614713,3.070175,i am a male of 40 years old. i have come to fi...
3178,Egoiste Platinum,Chanel,0.782224,1.000000,3.625803,3.205485,"it's fresh, and at the same time it reminds me..."
5237,Light Blue pour Homme,Dolce&Gabbana,1.000000,0.943320,2.884268,2.489353,reviewing after trying this on a strip and for...
1352,Acqua di Parma Colonia Assoluta,Acqua di Parma,0.958042,1.000000,3.012887,2.903696,"as a woman, i adore this fragrance and would d..."
2248,L’Eau d’Issey Pour Homme Sport,Issey Miyake,0.958146,1.000000,3.021944,2.665260,this one sadly doesn't work for me at all. at ...
4768,Colonia Futura,Acqua di Parma,1.000000,0.985294,2.790123,2.571839,"masculine, mature, old-school... but still a l..."


In [577]:
df_2.loc[262,'all_comments']

'if you can\'t afford oud minerale, then get the dua clone, which is a grand composition in its own right. walking around smelling like pickle juice is not the move. the most underrated aqua fragrances. just got the  ph and  . both are excellent excellent fragrances. love it... this is a very pleasant surprise. my initial impression on this one is beautiful; what a fresh, fruitier version of aqua! the salty aquatic notes are more subtle. i like this much more than  (too fishy) and original aqua. i do like atlantique and amara a lot, but   is more appealing to me, and has good performance, too. there is no egg smell in this one. a shame this was discontinued, i\'ll pick up any more that i find because this is a beauty for summer. an unknown and underrated gem. i just wrote a review on one of my all time faves - aqua - and despite my vast collection and attempts to find better in this category it\'s just hard to beat. \nthankfully reviews here are more upbeat for this version and having 

In [533]:
# x2 = vectorizer.transform()
dists, indices = nn2.kneighbors(X2[90])
df_2.iloc[indices[0]]

,fragrance,brand,votes_summer,votes_day,longevity,sillage,all_comments
621,Dior Homme Sport 2021,Dior,1.000000,0.950362,3.586139,2.748737,"i love it, for 30+ year old men. it reminds me..."
5237,Light Blue pour Homme,Dolce&Gabbana,1.000000,0.943320,2.884268,2.489353,reviewing after trying this on a strip and for...
2248,L’Eau d’Issey Pour Homme Sport,Issey Miyake,0.958146,1.000000,3.021944,2.665260,this one sadly doesn't work for me at all. at ...
5020,Viking Cologne,Creed,0.988848,0.951673,3.037152,2.431452,"one of the better releases of the new age, ..."
3285,Allure Homme Sport,Chanel,0.949691,1.000000,3.379836,2.897491,nice fresh fragrance with a creamy background....
5198,Euphoria Men,Calvin Klein,0.604608,1.000000,2.485623,2.188200,shaving cream. a whiff of ginger if i squint a...
2909,Royal Oud,Creed,0.247794,0.750849,3.547647,2.882666,just received a 5ml sample of this in the post...
685,Pasha de Edition Noire,Cartier,0.628070,0.894737,3.380435,2.889934,my batch was 5cab had very weak performance- 3...
4127,Colonia Intensa,Acqua di Parma,0.929059,1.000000,3.141968,2.728947,"it was not love at first sniff, but has worke..."
3399,Tom Ford for Men Tom Ford,Tom Ford,0.632514,1.000000,2.755165,2.606080,chnd wqt pysh ykh nskhh skht amrykh akhbnd pyd...


In [578]:
df_2.iloc[104,:]

fragrance                                           Y Eau Fraiche
brand                                          Yves Saint Laurent
votes_summer                                                  1.0
votes_day                                                0.891954
longevity                                                3.017467
sillage                                                  2.670103
all_comments    this stuff stinks of ginger. don't waste mone ...
Name: 729, dtype: object

In [138]:
pipe = Pipeline([('standardizer', scaler),
                 ('n_neighbors', nn)])

pipe.fit(df_1[features])

Pipeline(steps=[('standardizer', StandardScaler()),
                ('n_neighbors',
                 NearestNeighbors(algorithm='brute', n_neighbors=10))])

In [139]:
df_1[df_1['name'] == 'The Most Wanted']

,name,brand,votes_summer,votes_day,longevity,sillage
1773,The Most Wanted,Azzaro,0.143108,0.368745,3.791563,3.236853


In [546]:
df_1.iloc[847,]

fragrance                                      Leather Eau de Parfum
brand                                                 Acqua di Parma
votes_summer                                                0.153465
votes_day                                                   0.475248
longevity                                                   3.762646
sillage                                                     3.430034
summary_reviews    intense  note unique and satisfying scent eleg...
Name: 5154, dtype: object

In [141]:
np.array(df_1.loc[2310,:]).reshape(1, -1)

array([['Le Male Le Parfum', 'Jean Paul Gaultier', 0.197132, 0.491845,
        3.778210116731517, 2.934742871069949]], dtype=object)

In [142]:
dists, indices = pipe.named_steps['n_neighbors'].kneighbors(np.array(df_1[features].loc[1773,:]).reshape(1, -1))
df_1.iloc[indices[0]]

,name,brand,votes_summer,votes_day,longevity,sillage
1924,Lucky Charm,Carolina Herrera,1.000000,1.000000,4.750000,4.375000
2190,The Collection Daring Saffiano,Hugo Boss,0.625000,0.750000,4.222222,4.375000
2293,Kouros,Yves Saint Laurent,0.342949,0.739850,4.300243,3.993675
3983,Fleur du Mâle,Jean Paul Gaultier,0.538639,0.934256,4.193717,3.808671
2301,Jicky Limited Edition 2019,Guerlain,1.000000,1.000000,4.333333,3.693182
1469,Declaration Essence Limited Edition,Cartier,0.733333,1.000000,4.000000,3.839286
1549,Cuir de Russie,Guerlain,0.428571,0.857143,3.933333,3.964286
2564,CK One Collector's Bottle,Calvin Klein,0.500000,1.000000,4.000000,3.750000
867,Sauvage Elixir,Dior,0.249933,0.548044,4.450560,3.798338
838,Patchouli Elixir Precieux,Dior,0.333333,0.666667,4.000000,4.583333


In [78]:
np.array(df_1[features].iloc[0,:]).reshape(1, -1)

array([[0.882353  , 1.        , 3.19047619, 3.2       , 1.        ]])

In [27]:
indices

array([[1773, 1526, 3467, 2854, 1560, 2224, 3317,  264, 2201, 1335, 2002,
        2479, 3324, 3257, 1066, 1615, 1631, 2948,  199, 2651]])

In [333]:
vec_input = vectorizer.transform(["I want a sweet fragrance with great projection to wear at the club."])
dists, indices = nn.kneighbors(vec_input)
df_2.iloc[indices[0]]

,name,brand,votes_summer,votes_day,longevity,sillage,all_comments
3158,Azzaro Club Men,Azzaro,0.690476,0.968254,3.304569,2.916667,I purchased this fragrance for the cannabis no...
3865,Colonia C.L.U.B.,Acqua di Parma,0.975000,1.000000,2.989796,2.434211,Ordered Colonia Club in the green bottle and r...
1773,The Most Wanted,Azzaro,0.143108,0.368745,3.791563,3.236853,"Smells like sexy grape soda, would buy 8/10 do..."
2333,CK1 Palace,Calvin Klein,0.937500,1.000000,3.095238,2.857143,Sweet and gorgeous. Fruity as heck and the top...
3216,Silver Shadow,Davidoff,0.266667,1.000000,3.044643,2.702964,This is a bit of a hidden gem. I got this one ...
1747,La Nuit de l'Homme,Yves Saint Laurent,0.212755,0.351972,3.103788,2.576968,Hmmm I hear the ladies love this one... I'm no...
3339,Wanted,Azzaro,0.512069,0.946552,3.641686,3.238205,"its alright, smells similarly to a mix of ince..."
287,Obsessed for Men Calvin Klein,Calvin Klein,0.650273,1.000000,3.261029,2.866438,"A blind buy. I'm not one for sweet fragrances,..."
977,The Game Intense,Davidoff,0.142857,0.396825,3.505051,3.008130,Smell's like black afghano in the opening then...
586,V pour Homme,Valentino,0.177215,0.548523,3.532567,3.107798,A legendary fragrance. I still have a mostly f...


In [542]:
df[df['fragrance'] == 'Leather Eau de Parfum']

,fragrance,brand,gender,year,rating,count_ratings,accords,accord_opacity,accord_width,votes_have,...,pro_net_votes,con_net_votes,comment_users,comment_date,comments,url,filename,file_size,summary_reviews,all_comments
5154,Leather Eau de Parfum,Acqua di Parma,women and men,2019.0,4.02,412.0,"[leather, woody, citrus, animalic, fruity, smo...","[1.0, 0.923836, 0.737671, 0.707123, 0.682192, ...","[1.0, 0.911141, 0.69395, 0.6583100000000001, 0...",0.448606,...,"[11, 9, 7, 7, 7, 6, 6, 0]","[5, -1, -1, 0, -1, -3, -5, -6]",[],[],[],https://www.fragrantica.com/perfume/Acqua-di-P...,_Acqua-di-Parma-Leather Eau de Parfum.pkd,0.408821,intense note unique and satisfying scent eleg...,NaN


In [541]:
df_2[df_2['brand'] == "Acqua di Parma"]

,fragrance,brand,votes_summer,votes_day,longevity,sillage,all_comments
1352,Acqua di Parma Colonia Assoluta,Acqua di Parma,0.958042,1.000000,3.012887,2.903696,"as a woman, i adore this fragrance and would d..."
1741,Acqua di Parma Colonia,Acqua di Parma,1.000000,0.994714,2.790397,2.562204,i have collected over 200 fragrances and today...
2249,Colonia Ebano,Acqua di Parma,0.211382,0.788618,3.459119,3.038278,less similar to tf anthracite than i would hav...
2283,Note di Colonia II,Acqua di Parma,0.861111,0.972222,3.285714,2.979167,is a nice offering from . the cardamom a...
2976,Colonia Assoluta In Villa,Acqua di Parma,0.666667,1.000000,3.888889,2.937500,is a really lovely perfume. it projects mo...
3623,Colonia Club,Acqua di Parma,0.964657,1.000000,3.094406,2.788743,"i feel the mint, but i am not impressed. feels..."
3722,Colonia Essenza Special Edition 2015,Acqua di Parma,0.800000,1.000000,3.500000,3.500000,beautiful @josephazzato - no human being is re...
3893,Acqua di Parma Colonia Designer Edition,Acqua di Parma,0.600000,1.000000,3.200000,4.218750,"increble bottle, nice smell and luxury brand <3"
4127,Colonia Intensa,Acqua di Parma,0.929059,1.000000,3.141968,2.728947,"it was not love at first sniff, but has worke..."
4768,Colonia Futura,Acqua di Parma,1.000000,0.985294,2.790123,2.571839,"masculine, mature, old-school... but still a l..."


In [379]:
cond_gender = df['masculine'] > 3.5
cond_weather =(df['votes_summer'] > df['votes_winter'])
cond_day = (df['votes_day'] < df['votes_night'])

df[cond_gender & cond_weather & cond_day]

,name,brand,gender,year,rating,count_ratings,accords,accord_opacity,accord_width,votes_have,...,pro_net_votes,con_net_votes,comment_users,comment_date,comments,url,filename,ai_reviews,summary_reviews,all_comments
41,Emporio Armani For Him 2008,Giorgio Armani,men,2008.0,4.04,126.0,"[aromatic, citrus, woody, fresh spicy, musky, ...","[1.0, 0.966735, 0.762217, 0.63655, 0.629158, 0...","[1.0, 0.961191, 0.7225870000000001, 0.575975, ...",0.340299,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Giorgio-Ar...,_Giorgio-Armani-Emporio Armani For Him 2008.pkd,,NaN,NaN
265,Terre d'Hermes Flacon H 2020 Parfum,Hermès,men,2020.0,4.27,11.0,"[citrus, woody, mossy, mineral, earthy, fresh ...","[1.0, 0.832, 0.64, 0.58, 0.568, 0.544]","[1.0, 0.804, 0.58, 0.51, 0.496, 0.467999999999...",0.411765,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Hermes/Ter...,_Hermes-Terre d'Hermes Flacon H 2020 Parfum.pkd,,NaN,NaN
515,Pasha de Edition Noire,Cartier,men,2013.0,3.96,753.0,"[powdery, woody, fresh spicy, citrus, fresh, a...","[1.0, 0.968257, 0.961286, 0.930284, 0.870507, ...","[1.0, 0.962967, 0.9548340000000001, 0.91866500...",0.551940,...,"[5, 5, 2, 2, 2, 2, 1, -1]","[4, 4, 2, -2, -3, -2, -5, -6]","[konitz, PlatMik, FloridaFragLover, yutaka kob...","[2023-05-14T01:43:00.000000, 2023-05-01T18:14:...",[My batch was 5CAB had very weak performance- ...,https://www.fragrantica.com/perfume/Cartier/Pa...,_Cartier-Pasha de Cartier Edition Noire.pkd,Elegant Classy Delicate Light Versatile Well-b...,Elegant Classy Delicate Light Versatile Well-b...,My batch was 5CAB had very weak performance- 3...
818,Herrera for Men Sensual Vetiver Carolina Herrera,Carolina Herrera,men,2002.0,4.43,76.0,"[woody, aromatic, tobacco, earthy, warm spicy,...","[1.0, 0.88, 0.825316, 0.718987, 0.658228, 0.62...","[1.0, 0.86, 0.796202, 0.672152, 0.601266000000...",0.187970,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Carolina-H...,_Carolina-Herrera-Herrera for Men Sensual Veti...,,NaN,NaN
1018,Le Beau Le Parfum,Jean Paul Gaultier,men,2022.0,4.47,1569.0,"[sweet, woody, coconut, amber, vanilla, tropic...","[1.0, 0.960413, 0.880354, 0.853831, 0.765521, ...","[1.0, 0.9538150000000001, 0.8604120000000001, ...",0.447983,...,"[109, 105, 77, 62, 51, 48, 48, -33]","[47, -6, -16, -18, -40, -51, -57, -74]",[],[],[],https://www.fragrantica.com/perfume/Jean-Paul-...,_Jean-Paul-Gaultier-Le Beau Le Parfum.pkd,Pineapple and coconut scent gives a tropical v...,Pineapple and coconut scent gives a tropical v...,NaN
1242,Azzaro Pour Homme Limited Edition 2016,Azzaro,men,2016.0,4.35,31.0,"[citrus, lavender, woody, aromatic, fresh spicy]","[1.0, 0.957143, 0.807143, 0.783572, 0.747143]","[1.0, 0.95, 0.775, 0.7475, 0.705]",0.483146,...,[],[],"[Reza_2021, CGV, angelo..zucchinni, angelo..zu...","[2023-03-11T13:21:00.000000, 2023-02-02T06:03:...",[ykh `Tr m`tdl 4 fSl w shyryn b ryHh Glb sTwkh...,https://www.fragrantica.com/perfume/Azzaro/Azz...,_Azzaro-Azzaro Pour Homme Limited Edition 2016...,,NaN,ykh `Tr m`tdl 4 fSl w shyryn b ryHh Glb sTwkhd...
1284,Emporio Armani City Glam for Him Giorgio Armani,Giorgio Armani,men,2005.0,4.15,220.0,"[green, aromatic, citrus, fresh, earthy, warm ...","[1.0, 0.865101, 0.82953, 0.74094, 0.730201, 0....","[1.0, 0.8426170000000001, 0.801118, 0.697763, ...",0.350814,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Giorgio-Ar...,_Giorgio-Armani-Emporio Armani City Glam for H...,,NaN,NaN
1607,Jazz Prestige,Yves Saint Laurent,men,1993.0,4.51,164.0,"[fresh spicy, aromatic, woody, warm spicy, flo...","[1.0, 0.909328, 0.891107, 0.869631, 0.609762, ...","[1.0, 0.894216, 0.872958, 0.8479030000000001, ...",0.341797,...,[],[],"[sandmen, Jandp1, rcavs, lucky_ba, mapache, zo...","[2020-07-15T09:14:00.000000, 2020-02-03T09:46:...",[khlS@ ry'H@ llfndr lfrnsy lnqy m` bq@ mn l`sh...,https://www.fragrantica.com/perfume/Yves-Saint...,_Yves-Saint-Laurent-Jazz Prestige.pkd,,NaN,khlS@ ry'H@ llfndr lfrnsy lnqy m` bq@ mn l`shb...
2079,Declaration d'Un Soir,Cartier,men,2012.

In [405]:
df[(df['masculine']>3.5) &(df['votes_summer']<0.5) & (df['sillage'] > 3.5)]

,name,brand,gender,year,rating,count_ratings,accords,accord_opacity,accord_width,votes_have,...,pro_net_votes,con_net_votes,comment_users,comment_date,comments,url,filename,ai_reviews,summary_reviews,all_comments
1,The Collection Magnetic Musk,Hugo Boss,men,2021.0,4.18,11.0,"[beeswax, iris, powdery, honey, aromatic, vani...","[1.0, 1.0, 0.9475, 0.85, 0.8425, 0.7675, 0.745...","[1.0, 1.0, 0.93875, 0.825, 0.81625, 0.72875, 0...",0.072727,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Hugo-Boss/...,_Hugo-Boss-The Collection Magnetic Musk.pkd,,NaN,NaN
52,Ultra Male,Jean Paul Gaultier,men,2015.0,4.36,7983.0,"[vanilla, fruity, sweet, aromatic, cinnamon, w...","[1.0, 0.839852, 0.825061, 0.776103, 0.755886, ...","[1.0, 0.81316, 0.7959050000000001, 0.738787, 0...",0.607161,...,"[132, 126, 112, 84, 73, 70, 12, -24]","[98, 99, 74, 26, -15, -65, -77, -93]",[],[],[],https://www.fragrantica.com/perfume/Jean-Paul-...,_Jean-Paul-Gaultier-Ultra Male.pkd,Good longevity and projection Beautiful playfu...,Good longevity and projection Beautiful playfu...,NaN
302,Le Male Stimulating Body Spray 2006,Jean Paul Gaultier,men,2006.0,3.83,24.0,"[vanilla, lavender, aromatic, fresh spicy, pow...","[1.0, 0.8, 0.76, 0.695, 0.595, 0.565, 0.55, 0.54]","[1.0, 0.7666670000000001, 0.72, 0.644167, 0.52...",0.428571,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Jean-Paul-...,_Jean-Paul-Gaultier-Le Male Stimulating Body S...,,NaN,NaN
411,Polo,Ralph Lauren,men,1978.0,3.88,4613.0,"[woody, aromatic, fresh spicy, tobacco, earthy...","[1.0, 0.972226, 0.717688, 0.604385, 0.601834, ...","[1.0, 0.9675969999999999, 0.6706359999999999, ...",0.467945,...,"[42, 40, 39, 35, 27, 24, 20, 6]","[17, 15, 12, 0, -8, -9, -17, -23]",[],[],[],https://www.fragrantica.com/perfume/Ralph-Laur...,_Ralph-Lauren-Polo.pkd,Masculine scent Distinct and noticeable Classi...,Masculine scent Distinct and noticeable Classi...,NaN
448,Armani Code Le Parfum,Giorgio Armani,women,2008.0,3.83,64.0,"[citrus, warm spicy, white floral, rose, fresh...","[1.0, 0.941289, 0.886874, 0.829594, 0.757995, ...","[1.0, 0.931504, 0.868019, 0.8011929999999999, ...",0.257895,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Giorgio-Ar...,_Giorgio-Armani-Armani Code Le Parfum.pkd,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,La Collection Couturier Parfumeur Leather Oud,Dior,men,2010.0,4.09,854.0,"[animalic, leather, musky, oud, woody, amber, ...","[1.0, 0.930593, 0.790658, 0.769921, 0.749183, ...","[1.0, 0.919025, 0.7557680000000001, 0.731574, ...",0.394971,...,"[5, 4, 3, 2, 0, 0, 0, 0]","[2, 2, 1, 1, 1, 1, 1, 0]","[The Wolf, Arshia16, gustou, IamdrinkingBeer, ...","[2023-04-28T19:39:00.000000, 2022-08-19T03:35:...",[First I'll say that the comments are absolute...,https://www.fragrantica.com/perfume/Dior/La-Co...,_Dior-La Collection Couturier Parfumeur Leathe...,This is a fantastic fragrance for a manly man ...,This is a fantastic fragrance for a manly man ...,First I'll say that the comments are absolutel...
3620,Dior Homme Parfum,Dior,men,2014.0,4.52,5870.0,"[iris, woody, leather, powdery, violet, animal...","[1.0, 0.961336, 0.954783, 0.939811, 0.7, 0.677...","[1.0, 0.954892, 0.947247, 0.929779, 0.65, 0.62...",0.525558,...,"[138, 130, 131, 92, 93, 91, 89, 80]","[95, 48, 47, 40, -27, -34, -52, -54]",[Terry1978],[],[],https://www.fragrantica.com/perfume/Dior/Dior-...,_Dior-Dior Homme Parfum.pkd,Rich and daring blend Stunning release from Ch...,Rich and daring blend Stunning release from Ch...,NaN
3718,Bois Noir,Chanel,men,1987.0,4.38,48.0,"[woody, aromatic, floral, rose, lavender, musk...","[1.0, 0.838843, 0.756198, 0.747107, 0.738843, ...","[1.0, 0.811983, 0.715565, 0.704958, 0.695317, ...",0.125000,...,[],[],"[muzzbait, logica, silkconnection2, Perfumelov...","[2023-03-19T20:09:00.000000, 2023-01-05T03:59:...",[Quite an interesting and intriguing scent.\nT...,https://www.fragrantica.com/perfume/Chanel/Boi...,_Chanel-Bois Noir.pkd,,NaN,Quite 

In [406]:
df[(df['masculine']>3.5)&(df['sillage']>3.5)&(df['votes_day'] - 0.2 < df['votes_night'])]

,name,brand,gender,year,rating,count_ratings,accords,accord_opacity,accord_width,votes_have,...,pro_net_votes,con_net_votes,comment_users,comment_date,comments,url,filename,ai_reviews,summary_reviews,all_comments
1,The Collection Magnetic Musk,Hugo Boss,men,2021.0,4.18,11.0,"[beeswax, iris, powdery, honey, aromatic, vani...","[1.0, 1.0, 0.9475, 0.85, 0.8425, 0.7675, 0.745...","[1.0, 1.0, 0.93875, 0.825, 0.81625, 0.72875, 0...",0.072727,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Hugo-Boss/...,_Hugo-Boss-The Collection Magnetic Musk.pkd,,NaN,NaN
52,Ultra Male,Jean Paul Gaultier,men,2015.0,4.36,7983.0,"[vanilla, fruity, sweet, aromatic, cinnamon, w...","[1.0, 0.839852, 0.825061, 0.776103, 0.755886, ...","[1.0, 0.81316, 0.7959050000000001, 0.738787, 0...",0.607161,...,"[132, 126, 112, 84, 73, 70, 12, -24]","[98, 99, 74, 26, -15, -65, -77, -93]",[],[],[],https://www.fragrantica.com/perfume/Jean-Paul-...,_Jean-Paul-Gaultier-Ultra Male.pkd,Good longevity and projection Beautiful playfu...,Good longevity and projection Beautiful playfu...,NaN
302,Le Male Stimulating Body Spray 2006,Jean Paul Gaultier,men,2006.0,3.83,24.0,"[vanilla, lavender, aromatic, fresh spicy, pow...","[1.0, 0.8, 0.76, 0.695, 0.595, 0.565, 0.55, 0.54]","[1.0, 0.7666670000000001, 0.72, 0.644167, 0.52...",0.428571,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Jean-Paul-...,_Jean-Paul-Gaultier-Le Male Stimulating Body S...,,NaN,NaN
411,Polo,Ralph Lauren,men,1978.0,3.88,4613.0,"[woody, aromatic, fresh spicy, tobacco, earthy...","[1.0, 0.972226, 0.717688, 0.604385, 0.601834, ...","[1.0, 0.9675969999999999, 0.6706359999999999, ...",0.467945,...,"[42, 40, 39, 35, 27, 24, 20, 6]","[17, 15, 12, 0, -8, -9, -17, -23]",[],[],[],https://www.fragrantica.com/perfume/Ralph-Laur...,_Ralph-Lauren-Polo.pkd,Masculine scent Distinct and noticeable Classi...,Masculine scent Distinct and noticeable Classi...,NaN
448,Armani Code Le Parfum,Giorgio Armani,women,2008.0,3.83,64.0,"[citrus, warm spicy, white floral, rose, fresh...","[1.0, 0.941289, 0.886874, 0.829594, 0.757995, ...","[1.0, 0.931504, 0.868019, 0.8011929999999999, ...",0.257895,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Giorgio-Ar...,_Giorgio-Armani-Armani Code Le Parfum.pkd,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3718,Bois Noir,Chanel,men,1987.0,4.38,48.0,"[woody, aromatic, floral, rose, lavender, musk...","[1.0, 0.838843, 0.756198, 0.747107, 0.738843, ...","[1.0, 0.811983, 0.715565, 0.704958, 0.695317, ...",0.125000,...,[],[],"[muzzbait, logica, silkconnection2, Perfumelov...","[2023-03-19T20:09:00.000000, 2023-01-05T03:59:...",[Quite an interesting and intriguing scent.\nT...,https://www.fragrantica.com/perfume/Chanel/Boi...,_Chanel-Bois Noir.pkd,,NaN,Quite an interesting and intriguing scent.\nTh...
3725,Les Absolus d'Orient Oud Essentiel,Guerlain,women and men,2017.0,4.07,776.0,"[oud, leather, warm spicy, woody, amber, rose,...","[1.0, 0.879377, 0.747688, 0.711377, 0.702337, ...","[1.0, 0.8592730000000001, 0.7056359999999999, ...",0.474657,...,"[9, 7, 6, 2, 2, 2, 1, 1]","[7, -3, -3, -3, -3, -3, -4, -8]",[],[],[],https://www.fragrantica.com/perfume/Guerlain/L...,_Guerlain-Les Absolus d'Orient Oud Essentiel.pkd,"Exquisite luxury smell Dries down to a smoky, ...","Exquisite luxury smell Dries down to a smoky, ...",NaN
3726,212 Men Ice,Carolina Herrera,men,2010.0,4.44,75.0,"[fresh spicy, powdery, violet, woody, soft spi...","[1.0, 0.9904, 0.952, 0.8428, 0.832, 0.808, 0.6...","[1.0, 0.9887999999999999, 0.9440000000000001, ...",0.260870,...,[],[],[],[],[],https://www.fragrantica.com/perfume/Carolina-H...,_Carolina-Herrera-212 Men Ice.pkd,,NaN,NaN
3814,Reserve Collection: Italian Cypress,Tom Ford,women and men,2019.0,4.48,66.0,"[woody, aromatic, fresh spicy, green, citrus, ...","[1.0, 0.89374, 0.854695, 0.715321, 0.587809, 0...","[1.0, 0.87603, 0.8304779999999999, 0.667875, 0...",0.315789,...,[],[],"[Mez0Nez, Nate_915, TheArtsTraveler, MrBol